# Under Water Image Enhancement and Object Detection using MIRNet and YOLO

Image Enhancement Using MIRNet

In [ ]:
from google.colab import files # addressing upload and download operations
import os # operating system dependent functionality
import numpy as np # array operations
import keras # deep learning
from PIL import Image # Image processing

! pip install huggingface_hub
from huggingface_hub import from_pretrained_keras # download the model

model = from_pretrained_keras("keras-io/lowlight-enhance-mirnet", compile=False)

def create_enhanced_image(sample):
  low_light_img = Image.open(sample).convert('RGB')
  image = keras.preprocessing.image.img_to_array(low_light_img)
  image = image.astype('float32') / 255.0
  image = np.expand_dims(image, axis = 0)
  output = model.predict(image) # model inference to enhance the low light pics
  output_image = output[0] * 255.0
  output_image = output_image.clip(0,255)
  output_image = output_image.reshape((np.shape(output_image)[0],np.shape(output_image)[1],3))
  output_image = np.uint32(output_image)
  FinalOutput = Image.fromarray(output_image.astype('uint8'),'RGB')
  FinalOutput.save(sample)

Cloning Darknet backbone for YOLOv3

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

Cloning supporting files for detection

In [ ]:
os.chdir("/content")
!git clone https://Soumyaratna_Debnath@bitbucket.org/soumyaratnadebnath/uwod-support-files.git

for item in os.listdir('./uwod-support-files/'):
    os.rename('./uwod-support-files/'+item, './'+item)

<hr>

Upload Custom Image for Detection

In [ ]:
print("Please Upload a .JPG Image for Detection")
sample = files.upload()
source = list(sample.keys())[0]
os.rename("./"+source, "./sample.jpg")
create_enhanced_image("sample.jpg")
os.rename("./sample.jpg", "./data/sample.jpg")

Performing Underwater Object Detetction

In [ ]:
!chmod +x darknet/darknet
!darknet/darknet detector test custom_data/labelled_data.data yolov3_custom.cfg yolov3_trained_model.weights data/sample.jpg -thresh 0.3

Displaying and downloading the results

In [ ]:
# helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

imShow('predictions.jpg')
download('predictions.jpg')